In [10]:
#function to test models 
import nest_asyncio
nest_asyncio.apply()

In [20]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.dataloader.loader import DataLoader

In [21]:
k = DataLoader('../scripts/model_load/loader_y.yaml')

In [22]:
k.data_dict

{'data': {'from_datasets': 'CIFAR10',
  'from_file': False,
  'work_dir': '/home/malav/mb_pytorch/data/',
  'file': '/home/malav/mb_pytorch/data/csv/file_data.csv',
  'use_img_dir': True,
  'img_dir': '/home/malav/mb_pytorch/data/images1/',
  'thresholding_pd': 0},
 'train_params': {'batch_size': 4,
  'num_workers': 4,
  'pin_memory': True,
  'shuffle': True,
  'seed': 42},
 'test_params': {'batch_size': 4,
  'num_workers': 4,
  'pin_memory': True,
  'shuffle': True,
  'seed': 42},
 'transforms_list': {'transform': True,
  'resize': {'val': False, 'args': {'size': [256, 256]}},
  'random_crop': {'val': False, 'args': {'size': [224, 224]}},
  'random_horizontal_flip': {'val': True, 'args': {'p': 0.5}},
  'random_vertical_flip': {'val': False, 'args': {'p': 0.5}},
  'random_rotation': {'val': True, 'args': {'degrees': 90}},
  'random_grayscale': {'val': False, 'args': {'p': 0.1}},
  'random_color_jitter': {'val': True,
   'args': {'brightness': 0.4,
    'contrast': 0.4,
    'saturation':

In [23]:
model_data = ModelLoader(k.data_dict['model'])

In [24]:
model_data.get_model()

AttributeError: module 'torchvision.models' has no attribute 'model_name'